In [1]:
from surprise import Dataset , Reader
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('deep_data.csv')
df.reset_index(inplace = True , drop = True)
df = df[['product_id','category_index','seller_index']]
df

,product_id,category_index,seller_index
0,1e9e8ef04dbcff4541ed26657ea517e5,0,438
1,6a2fb4dd53d2cdb88e0432f1284a004c,0,2973
2,6a2fb4dd53d2cdb88e0432f1284a004c,0,2973
3,0d009643171aee696f4733340bc2fdd0,0,998
4,0d009643171aee696f4733340bc2fdd0,0,998
...,...,...,...
112645,10dbe0fbaa2c505123c17fdc34a63c56,74,115
112646,bd2ada37b58ae94cc838b9c0569fecd8,74,1558
112647,fa51e914046aab32764c41356b9d4ea4,74,2854
112648,fa51e914046aab32764c41356b9d4ea4,74,2854


In [8]:
df1 = pd.read_csv('ratings.csv')
df1

,product_id,review_score
0,fd25ab760bfbba13c198fa3b4f1a0cd3,4.666667
1,be0dbdc3d67d55727a65d4cd696ca73c,3.764706
2,d1c427060a0f73f6b889a5c7c61f2ac4,4.213622
3,52c80cedd4e90108bf4fa6a206ef6b03,4.157895
4,3880d25d502b15b1de6fddc42ad1d67a,5.000000
...,...,...
32946,2381a3052dd37f97e616bf160467acae,5.000000
32947,6e29f8de1ad465c1fa275aae299fa98b,5.000000
32948,6160c6145b50c401702912baf0fd7de5,5.000000
32949,4fb209073dc2ab6a6d3c259a7c285501,5.000000


In [47]:
data = pd.merge(df ,df1)
data

,product_id,category_index,seller_index,review_score
0,1e9e8ef04dbcff4541ed26657ea517e5,0,438,5.0
1,6a2fb4dd53d2cdb88e0432f1284a004c,0,2973,3.5
2,6a2fb4dd53d2cdb88e0432f1284a004c,0,2973,3.5
3,0d009643171aee696f4733340bc2fdd0,0,998,3.7
4,0d009643171aee696f4733340bc2fdd0,0,998,3.7
...,...,...,...,...
112645,10dbe0fbaa2c505123c17fdc34a63c56,74,115,2.0
112646,bd2ada37b58ae94cc838b9c0569fecd8,74,1558,5.0
112647,fa51e914046aab32764c41356b9d4ea4,74,2854,5.0
112648,fa51e914046aab32764c41356b9d4ea4,74,2854,5.0


In [35]:
vector = data[['category_index','seller_index','review_score']].values
item_ids = data.index
product_id = data.product_id


In [26]:
from sklearn.metrics.pairwise import cosine_similarity
import scipy

In [27]:
def get_product(item_id):
    idx = item_ids[item_id]
    item_profile = vector[idx:idx+1]
    return item_profile
def get_related_products(ids):
    item_profile_list = [get_product(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profile_list)
    return item_profiles

In [69]:
class ContentBasedRecommender:
    MODEL_NAME = 'Content-Based'
    def __init__(self,items_df=None,item_ids=None):
        self.item_ids = item_ids
        self.item_df = data
    def get_model_name(self):
        return self.MODEL_NAME
    def _get_similar_products(self, item_id, topn=1000):
        cosine_similarities = cosine_similarity(get_product(item_id),vector)
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        similar_products = sorted([(item_ids[i],cosine_similarities[0,i]) for i in similar_indices ], key=lambda x : -x[1])
        return similar_products
    def get_recommendation(self,item_id,items_to_ignore = [], topn=10, verbose = False):
        similar_items = self._get_similar_products(item_id)
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        return similar_items_filtered[:10]

In [70]:
csm = ContentBasedRecommender(data,item_ids)
csm.get_recommendation(5)


[(9, 1.0),
 (8, 1.0),
 (7, 1.0),
 (6, 1.0),
 (5, 1.0),
 (4, 1.0),
 (3, 1.0),
 (10, 1.0),
 (647, 0.9999999994422307),
 (1247, 0.9999999994422307)]